In [13]:
import numpy as np 
import pandas as pd 
import re
from bs4 import BeautifulSoup
import nltk
from sklearn.ensemble import RandomForestClassifier

In [14]:
df = pd.read_csv('labelledTrainData.tsv', header=0, delimiter="\t", quoting =3)

In [15]:
df.shape

(25000, 3)

In [16]:
df.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [17]:
def review_to_words(raw_reviews):
    review_text = BeautifulSoup(raw_reviews).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words=[w for w in words if not w in stops ]
    return(" ".join(meaningful_words))

In [18]:
num_reviews = df["review"].size
clean_train_reviews = []

for i in range( 0, num_reviews):
        clean_train_reviews.append( review_to_words( df["review"][i]))

In [19]:
num_rev = df["review"].size

In [20]:
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [22]:
stopwords_eng = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub("[^a-z]", " ", text)
    words = [word for word in text.split() if word not in stopwords_eng]
    text = " ".join(words)
    return text

df = df.map(clean_text)
df.head()

NameError: name 'X_train' is not defined

In [23]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 150) 
forest = forest.fit( train_data_features, df["sentiment"] )

In [24]:
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

num_reviews = len(test["review"])
clean_test_reviews = [] 


for i in range(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [25]:
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "submit.csv", index=False, quoting=3 )
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",1
4,"""12128_7""",1
